In [1]:
import pandas as pd
import string
import re
import numpy as np

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec, KeyedVectors

In [5]:
questions = pd.read_csv("./data/labelled_clean_questions.csv")

In [49]:
train_balanced = questions[questions['target'] == 1]
train_balanced = train_balanced.append(questions[questions['target'] == 0].sample(n = len(train_balanced))).reset_index(drop = True)
train_balanced['target'].value_counts()

balanced_text = train_balanced["clean_tokenized"].values.astype('U')

train_text, test_text, target_train, target_test = train_test_split(balanced_text, train_balanced["target"], test_size=0.25)


In [50]:
vectorizer = TfidfVectorizer(use_idf=True, ngram_range=(1,2))
tfidf_model = vectorizer.fit(train_text)
train_tfidf = tfidf_model.transform(train_text)
test_tfidf = tfidf_model.transform(test_text)

In [36]:
from sklearn.linear_model import SGDClassifier

In [51]:
clf = SGDClassifier().fit(train_tfidf, target_train)

In [95]:
preds = clf.predict(test_tfidf)

In [96]:
print(classification_report(target_test, preds))

              precision    recall  f1-score   support

           0       0.69      0.67      0.68     20230
           1       0.68      0.70      0.69     20175

    accuracy                           0.68     40405
   macro avg       0.68      0.68      0.68     40405
weighted avg       0.68      0.68      0.68     40405



In [107]:
rf = RandomForestClassifier(max_depth= 20, min_samples_leaf= 3,
                         min_samples_split= 5, n_estimators= 2000, max_features="auto")
rf.fit(train_tfidf, target_train)

rf_preds = rf.predict(test_tfidf)

Significance T-Testing

In [103]:
from scipy.stats import wilcoxon

In [108]:
wilcoxon(rf_preds, preds)

WilcoxonResult(statistic=29158452.0, pvalue=1.4461457700622444e-84)

In [130]:
!pip install pickle-mixin
import pickle


In [131]:
with open('tfidf_random_forest.pkl','wb') as f:
    pickle.dump(rf,f)

In [132]:
with open('base.pkl','wb') as f:
    pickle.dump(clf,f)